In [2]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
import winsound

# Load face detector and facial landmarks model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Thresholds
EAR_THRESHOLD = 0.25
FRAME_THRESHOLD = 10
HEAD_TILT_THRESHOLD = 15  # Max head tilt angle allowed
frame_count = 0
drowsy_alarm_on = False
distraction_alarm_on = False

# Define 3D model points (Reference facial landmarks for head pose)
model_points = np.array([
    (0.0, 0.0, 0.0),    # Nose tip
    (0.0, -330.0, -65.0),  # Chin
    (-225.0, 170.0, -135.0),  # Left eye corner
    (225.0, 170.0, -135.0),   # Right eye corner
    (-150.0, -150.0, -125.0),  # Left mouth corner
    (150.0, -150.0, -125.0)   # Right mouth corner
], dtype="double")

# Camera parameters (Assumed values for a standard webcam)
focal_length = 640  # Approximate focal length
center = (320, 240)  # Image center
camera_matrix = np.array([
    [focal_length, 0, center[0]],
    [0, focal_length, center[1]],
    [0, 0, 1]
], dtype="double")

dist_coeffs = np.zeros((4, 1))  # No lens distortion

# Function to calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Start video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Extract eye landmarks
        left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
        right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])

        # Compute EAR
        left_EAR = eye_aspect_ratio(left_eye)
        right_EAR = eye_aspect_ratio(right_eye)
        avg_EAR = (left_EAR + right_EAR) / 2.0

        # Extract head pose landmarks
        image_points = np.array([
            (landmarks.part(30).x, landmarks.part(30).y),  # Nose tip
            (landmarks.part(8).x, landmarks.part(8).y),    # Chin
            (landmarks.part(36).x, landmarks.part(36).y),  # Left eye corner
            (landmarks.part(45).x, landmarks.part(45).y),  # Right eye corner
            (landmarks.part(48).x, landmarks.part(48).y),  # Left mouth corner
            (landmarks.part(54).x, landmarks.part(54).y)   # Right mouth corner
        ], dtype="double")

        # Solve PnP for head pose estimation
        success, rotation_vector, translation_vector = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs)

        # Convert rotation vector to Euler angles (Yaw, Pitch, Roll)
        rmat, _ = cv2.Rodrigues(rotation_vector)
        angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)

        yaw, pitch, roll = angles  # Extract angles

        # Draw eye contours
        cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
        cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

        # Display head pose angles
        cv2.putText(frame, f"Yaw: {int(yaw)}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        cv2.putText(frame, f"Pitch: {int(pitch)}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        # Drowsiness Detection
        if avg_EAR < EAR_THRESHOLD:
            frame_count += 1
            if frame_count >= FRAME_THRESHOLD:
                cv2.putText(frame, "DROWSINESS ALERT!", (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                if not drowsy_alarm_on:  # Play sound only once
                    winsound.Beep(1000, 500)  # Beep sound (Windows)
                    drowsy_alarm_on = True  # Set alarm flag
        else:
            frame_count = 0
            drowsy_alarm_on = False  # Reset flag when eyes open

        # Distraction Detection (if head is tilted for too long)
        if abs(yaw) > HEAD_TILT_THRESHOLD:
            cv2.putText(frame, "LOOKING AWAY ALERT!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            if not distraction_alarm_on:
                winsound.Beep(800, 500)  # Different beep sound
                distraction_alarm_on = True  # Set alarm flag
        else:
            distraction_alarm_on = False  # Reset flag when head is straight

    # Display frame
    cv2.imshow("Drowsiness & Distraction Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()